In [1]:
import logging

logging.basicConfig(level=logging.INFO)
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
from eo_tools.S1.process import process_insar
import eo_tools
from eodag import EODataAccessGateway
import shapely
import json

dag = EODataAccessGateway()
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

INFO:eodag.config:Loading user configuration from: /home/figi/.config/eodag/eodag.yml
INFO:eodag.core:usgs: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:aws_eos: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:meteoblue: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:hydroweb_next: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:wekeo: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:creodias_s3: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:Locations configuration loaded from /home/figi/.config/eodag/locations.yml


In [3]:
data_dir = "data"

ids = [
    "S1A_IW_SLC__1SDV_20230906T061510_20230906T061537_050203_060ADE_A7A1",
    "S1A_IW_SLC__1SDV_20230909T175013_20230909T175040_050254_060CA3_6847",
]

primary_dir = f"{data_dir}/{ids[0]}.zip"
secondary_dir = f"{data_dir}/{ids[1]}.zip"
outputs_prefix=f"{data_dir}/res/test-full-processor-hv"

In [4]:
bbox = {"lonmin": 0.08905898091569497, "latmin": 52.15527412683906, "lonmax": 0.9565339502005088, "latmax": 52.69722175598818}



search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2023-09-03",
    "end": "2023-09-17",
    "geom": bbox
}

results, _ = dag.search(**search_criteria)
# print(json.dumps(results.as_geojson_object(), indent=2))
to_dl = [it for it in results if it.properties["id"] in ids]
print(f"{len(to_dl)} products to download")
dag.download_all(to_dl, outputs_prefix=data_dir, extract=False)

INFO:eodag.core:Searching product type 'S1_SAR_SLC' on provider: cop_dataspace
INFO:eodag.search.qssearch:Sending search request: http://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel1/search.json?startDate=2023-09-03&completionDate=2023-09-17&geometry=POLYGON ((0.0891 52.1553, 0.0891 52.6972, 0.9565 52.6972, 0.9565 52.1553, 0.0891 52.1553))&productType=SLC&maxRecords=20&page=1&exactCount=1
INFO:eodag.core:Found 7 result(s) on provider 'cop_dataspace'
INFO:eodag.core:Downloading 2 products


2 products to download


Downloaded products:   0%|          | 0/2 [00:00<?, ?product/s]

0.00B [00:00, ?B/s]

INFO:eodag.download.base:Download url: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(e8c9b1e6-2045-4cdb-849b-e476b30014aa)/$value
INFO:eodag.download.base:Product already downloaded: /home/figi/software/work/eodh/user-workflows/test-1/data/S1A_IW_SLC__1SDV_20230909T175013_20230909T175040_050254_060CA3_6847.zip
INFO:eodag.download.base:Extraction not activated. The product is available as is.
INFO:eodag.product:Remote location of the product is still available through its 'remote_location' property: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(e8c9b1e6-2045-4cdb-849b-e476b30014aa)/$value


0.00B [00:00, ?B/s]

INFO:eodag.download.base:Download url: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(d85543f6-d8f5-4db5-8e57-90bdda20b734)/$value
INFO:eodag.download.base:Product already downloaded: /home/figi/software/work/eodh/user-workflows/test-1/data/S1A_IW_SLC__1SDV_20230906T061510_20230906T061537_050203_060ADE_A7A1.zip
INFO:eodag.download.base:Extraction not activated. The product is available as is.
INFO:eodag.product:Remote location of the product is still available through its 'remote_location' property: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(d85543f6-d8f5-4db5-8e57-90bdda20b734)/$value


['/home/figi/software/work/eodh/user-workflows/test-1/data/S1A_IW_SLC__1SDV_20230909T175013_20230909T175040_050254_060CA3_6847.zip',
 '/home/figi/software/work/eodh/user-workflows/test-1/data/S1A_IW_SLC__1SDV_20230906T061510_20230906T061537_050203_060ADE_A7A1.zip']

In [5]:
shp = shapely.box(bbox["lonmin"], bbox["latmin"], bbox["lonmax"], bbox["latmax"])


In [6]:
eo_tools.util.explore_products(results, shp)

In [7]:
out_dir = process_insar(
    dir_prm=primary_dir,
    dir_sec=secondary_dir,
    outputs_prefix=outputs_prefix,
    aoi_name=None,
    shp=shp,
    pol="vv",
    subswaths=["IW1", "IW2", "IW3"],
    write_coherence=True,
    write_interferogram=True,
    write_primary_amplitude=True,
    write_secondary_amplitude=False,
    apply_fast_esd=True,
    dem_upsampling=1.8,
    dem_force_download=False,
    dem_buffer_arc_sec=40,
    boxcar_coherence=[3, 3],
    filter_ifg=True,
    multilook=[1, 4],
    warp_kernel="bicubic",
    clip_to_shape=True,
)

RuntimeError: Images must be from the same relative orbit.